In [1]:
# https://wikidocs.net/231156 -> LLM 구성하기
# https://wikidocs.net/196075 -> OpenAI API Key 발급받는법

In [2]:
# # LangChain을 설치하면 langchain-core, langchain-community, langsmith 등이 함께 설치되어 프로젝트 수행에 필수적인 라이브러리들은 한번에 설치됩니다.
# !pip install langchain

# # GPT-3.5, GPT-4 등 LLM 모델과 기타 보조 도구를 포함
# !pip install langchain_openai

# # OpenAI 모델이 사용하는 토크나이저(Tokenizer)
# !pip install tiktoken

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
import os
api_key = os.environ.get('OPENAI_API_KEY')
print(api_key)

In [5]:
from langchain_openai import ChatOpenAI

# model
llm = ChatOpenAI(model="gpt-4o-mini")

# chain 실행
llm.invoke("지구의 자전 주기는?")

AIMessage(content='지구의 자전 주기는 약 24시간입니다. 정확하게는 23시간 56분 4초 정도로, 이를 "항성일"이라고 합니다. 그러나 일상적으로 우리가 사용하는 24시간은 태양일로, 태양이 하늘에서 동일한 위치에 다시 나타날 때까지의 시간을 기준으로 합니다. 태양일은 항성일보다 약 4분 더 길기 때문에 24시간으로 설정되어 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 15, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'id': 'chatcmpl-BThwmRS9jhvbEkWkdq3RjJdMsqc2C', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6b6305c8-282a-4379-8f50-ae61c1b2dfc0-0', usage_metadata={'input_tokens': 15, 'output_tokens': 98, 'total_tokens': 113, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in astronomy. Answer the question. <Question>: {input}'), additional_kwargs={})])

In [7]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

# chain 연결 (LCEL)
chain = prompt | llm

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

AIMessage(content="지구의 자전 주기는 약 24시간입니다. 더 정확하게 말하자면, 지구가 자기 축을 한 바퀴 도는 데 걸리는 시간은 약 23시간 56분 4초로, 이를 '항성일'이라고 합니다. 하지만 우리가 일반적으로 사용하는 24시간은 태양이 하늘에서 이동하는 것을 기준으로 한 '태양일'입니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 29, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'id': 'chatcmpl-BThwqlN7fE4XqyvLVx49uXUaI0QRN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c02bff3a-5025-475b-b13f-511f730ad0e6-0', usage_metadata={'input_tokens': 29, 'output_tokens': 88, 'total_tokens': 117, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
llm = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

'지구의 자전 주기는 약 24시간입니다. 정확히는 23시간 56분 4초, 즉 약 86,164초로, 이를 "항성일"이라고 합니다. 그러나 우리가 사용하는 일상적인 하루는 태양이 하늘에서 같은 위치에 돌아오는 주기인 "태양일"로, 약 24시간입니다. 태양일은 태양의 위치 변화를 기준으로 한 것이기 때문에 지구의 자전 속도와 태양과의 상대적 움직임에 따라 약간의 차이가 있습니다.'

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser

# 전문가 분야와 질문 입력 받기
expert_input = input("전문가 분야를 입력하세요 (예: medicine, AI, law 등): ")
user_input = input("질문을 입력하세요: ")

# 템플릿 구성 --- LLM에서의 {} 표시는 문자열 변수 : 여기에 값을 채우라는 의미.
prompt = ChatPromptTemplate.from_template(
    "You are an expert in {expertise}. Answer the question.\n<Question>: {question}"
)

# 모델 및 출력 파서 구성
llm = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

# 체인 구성
chain = prompt | llm | output_parser

# 실행
response = chain.invoke({
    "expertise": expert_input,
    "question": user_input
})

print("전문가 분야를 입력하세요 (예: medicine, AI, law 등):", expert_input)
print("질문을 입력하세요:", user_input)
print("----------")
print(response)

전문가 분야를 입력하세요 (예: medicine, AI, law 등):  AI
질문을 입력하세요:  ChatGPT 말고 추천할만한 OpenAI에는 뭐가 있을까?


전문가 분야를 입력하세요 (예: medicine, AI, law 등): AI
질문을 입력하세요: ChatGPT 말고 추천할만한 OpenAI에는 뭐가 있을까?
----------
OpenAI는 ChatGPT 외에도 여러 가지 흥미로운 제품과 기술을 개발하고 있습니다. 다음은 추천할 만한 OpenAI의 몇 가지 프로젝트입니다:

1. **DALL-E**: 이미지 생성 모델로, 사용자가 입력한 텍스트 설명을 기반으로 독창적인 이미지를 생성합니다. 창의적인 시각적 콘텐츠 생성에 매우 유용합니다.

2. **Codex**: 프로그래밍 코드 작성과 관련된 AI 모델로, 자연어로 된 설명을 코드로 변환합니다. GitHub Copilot으로도 유명하며, 개발자들이 코드를 더 쉽게 작성하는 데 도움을 줍니다.

3. **Whisper**: 오디오 인식 모델로, 음성을 텍스트로 변환하는 데 사용됩니다. 다양한 언어와 악센트를 지원하여, 여러 비즈니스 및 개인 용도로 활용할 수 있습니다.

OpenAI의 다양한 도구와 모델들은 창의성, 생산성, 코드 작성 등 다양한 분야에서 유용하게 사용될 수 있습니다. 각 제품의 특성과 용도를 고려하여 선택하면 좋습니다.
